### This notebook presents an execution tree enhanced with provenance.
### In this view, ALL of the dependencies are presented in the tree. None simplification or cleaning is performed
### The grey objects in the graph are Evaluations that are NOT function calls

In [10]:
# Imports
import sqlite3

from graphviz import Graph

from debugprov.validity import Validity
from debugprov.node import Node
from debugprov.execution_tree import ExecutionTree
from debugprov.execution_tree_creator import ExecTreeCreator
from debugprov.top_down import TopDown
from debugprov.heaviest_first import HeaviestFirst
from debugprov.visualization import Visualization
from debugprov.provenance_enhancement import ProvenanceEnhancement 
from debugprov.single_stepping import SingleStepping
from debugprov.divide_and_query import DivideAndQuery

In [11]:
NOW2_SQLITE_PATH = 'C:/Users/linha/Desktop/example/.noworkflow/db.sqlite'

In [12]:
CURSOR = sqlite3.connect(NOW2_SQLITE_PATH).cursor()

In [13]:
creator = ExecTreeCreator(CURSOR)
exec_tree = creator.create_exec_tree()

In [14]:
prov = ProvenanceEnhancement(exec_tree, CURSOR)

In [15]:
len(prov.dependencies)

27

In [17]:
vis = Visualization(prov.exec_tree)

In [18]:
vis.view_exec_tree('exec_tree_prov_full')